In [ ]:
!pip install MetaTrader5
!pip install ta

In [ ]:
# connect to MetaTrader 5

import pandas as pd
import tensorflow as tf
import os
try:
    import MetaTrader5 as mt5
    if not mt5.initialize():
        print("initialize() failed")
        mt5.shutdown()
        gold_data = pd.read_csv("gold_data.csv")
    else:
        # request connection status and parameters
        print(mt5.terminal_info())
        # get data on MetaTrader 5 version
        print(mt5.version())
        gold_ticks = mt5.copy_rates_from_pos("GOLD", mt5.TIMEFRAME_M5, 0, 100000)
        # shut down connection to MetaTrader 5
        mt5.shutdown()
        gold_data = pd.DataFrame(gold_ticks)
        gold_data.to_csv("gold_data.csv")
except Exception as e:
    print(e)
    if os.path.exists("gold_data.csv"):
        gold_data = pd.read_csv("gold_data.csv")
    else:
        print("gold_data.csv not found, no way to retrieve the data. Exiting...")
        exit()